## Import modules

In [1]:
classpath.addPath("tinyir-1.1.jar")

In [2]:
import scala.xml.XML
import ch.ethz.dal.tinyir._
import com.github.aztek.porterstemmer.PorterStemmer

import scala.xml.XML
import ch.ethz.dal.tinyir._
import com.github.aztek.porterstemmer.PorterStemmer

In [3]:
// import scala.io.Source  // for importing txt files
import java.io._  // for saving txt files
// import scala.collection.mutable.HashMap  //HashMap used for counting elements in linear time

import java.io._

In [4]:
// import scala.util.Random
import scala.collection.mutable.{Map => MutMap}
// enables "mutable lists"
import scala.collection.mutable.ListBuffer  

import scala.collection.mutable.{Map => MutMap}
import scala.collection.mutable.ListBuffer

## Define classes and functions

In [5]:
def token_filter(text_body: String) = {
    processing.StopWords.filterOutSW(
        processing.Tokenizer.tokenize(text_body.
                                      replaceAll("\\P{L}+", " "))
    ).
    map(x => PorterStemmer.stem(x)).filter(_.trim.nonEmpty).toList
}

defined function token_filter

In [6]:
// Create YMParse for customized tokenization

class YMParse(is: InputStream) extends processing.TipsterParse(is) { 
    override def tokens: List[String] = token_filter(content)
    def hname: Int = name.hashCode()
    def htokens: List[Int] = token_filter(content).map(t => t.hashCode())
  //override def ID: String = name - need to fix the ID.
}

defined class YMParse

In [7]:
import ch.ethz.dal.tinyir.processing.TipsterParse
import ch.ethz.dal.tinyir.processing.Tokenizer
import ch.ethz.dal.tinyir.processing.XMLDocument

// Create YMStream for customized stream using YMParse
class YMStream (path: String, ext: String = "") extends io.ParsedXMLStream(new io.ZipDirStream(path, "")){
  def stream : Stream[YMParse] = unparsed.stream.map(is => new YMParse(is))
  def length = unparsed.length 
}



import ch.ethz.dal.tinyir.processing.TipsterParse
import ch.ethz.dal.tinyir.processing.Tokenizer
import ch.ethz.dal.tinyir.processing.XMLDocument
defined class YMStream

In [9]:
val collection = new YMStream("resources").stream
val collection2 = new io.TipsterStream("resources").stream

collection: Stream[YMParse] = Stream(
  cmd5$$user$YMParse@34530c36,
  cmd5$$user$YMParse@2398123c,
  cmd5$$user$YMParse@64cae4d8,
  cmd5$$user$YMParse@3e78ec13,
  cmd5$$user$YMParse@3de035b6,
  cmd5$$user$YMParse@1955077a,
  cmd5$$user$YMParse@45e2eeba,
  cmd5$$user$YMParse@5c2e5c04,
  cmd5$$user$YMParse@29e1cde3,
  cmd5$$user$YMParse@6f0443a1,
  cmd5$$user$YMParse@4b0d621a,
  cmd5$$user$YMParse@4eb6856a,
  cmd5$$user$YMParse@7be0e7dd,
  cmd5$$user$YMParse@1f8c82aa,
  cmd5$$user$YMParse@f7a8b7a,
  cmd5$$user$YMParse@5b20df9d,
  cmd5$$user$YMParse@debc803,
  cmd5$$user$YMParse@7793579d,
  cmd5$$user$YMParse@3f4c0e28,
...
collection2: Stream[XMLDocument] = Stream(
  ch.ethz.dal.tinyir.processing.TipsterParse@4ccdac19,
  ch.ethz.dal.tinyir.processing.TipsterParse@6bdbbabe,
  ch.ethz.dal.tinyir.processing.TipsterParse@3bc0b731,
  ch.ethz.dal.tinyir.processing.TipsterParse@17ad9b0e,
  ch.ethz.dal.tinyir.processing.TipsterParse@40bdad4b,
  ch.ethz.dal.tinyir.processing.TipsterParse@4ff4a3ec

In [ ]:
import ch.ethz.dal.tinyir.processing._
def fwStream (docs: Stream[YMParse]) :
   Stream[(Int,List[Int])]
   = docs.map(d => (d.hname, d.htokens))

In [ ]:
def fwIndex (docs: Stream[YMParse]) :
   Map[Int,List[Int]] = fwStream(docs).toMap

In [8]:
// hashed version
def postings (s: Stream[YMParse]): Stream[(Int,Int)] =
  s.flatMap( d => d.htokens.map(token => (token,d.hname)) )
var invIndex = scala.collection.mutable.Map[Int, List[Int]]()

defined function postings
invIndex: collection.mutable.Map[Int, List[Int]] = Map()

In [ ]:
// unhashed verison
def postings (s: Stream[YMParse]): Stream[(String,String)] =
  s.flatMap( d => d.tokens.map(token => (token,d.name)) )
var invIndex = scala.collection.mutable.Map[String, List[String]]()

In [ ]:
val posting1 = postings(collection.take(100)).groupBy(_._1).mapValues(_.map(p => p._2).distinct.sorted)

In [ ]:
// create twenty sets from original collection (or another number)
// create posting for each of these sets
// then call inverted index dictionary. for each token, call get or else = empty list

In [ ]:
for( a <- (5000 to 100000 by 5000))
{
    var imd_posting = postings(collection.slice(a-5000,a)).groupBy(_._1).mapValues(_.map(p => p._2).distinct.sorted)
    for (key <- imd_posting.keys){
        invIndex(key) = invIndex.getOrElse(key,List()).++(imd_posting(key))
    }
    print(a)
    //invIndex = (invIndex.toSeq ++ imd_posting.toSeq)
    //.groupBy{case(token,id) => token}
    //.mapValues(item => item.flatMap{case(token,id) => id}.toList)
}

500010000

In [ ]:
for( a <- (5000 to 100000 by 5000))
{
    var imd_posting = postings(collection.slice(a-5000,a)).groupBy(_._1).mapValues(_.map(p => p._2).distinct.sorted)
    for (key <- imd_posting.keys){
        invIndex(key) = invIndex.getOrElse(key,List()).++(imd_posting(key))
    }
    print(a)
    //invIndex = (invIndex.toSeq ++ imd_posting.toSeq)
    //.groupBy{case(token,id) => token}
    //.mapValues(item => item.flatMap{case(token,id) => id}.toList)
}

In [ ]:
invIndex.values.flatten.toList.

In [ ]:
postings(collection.take(1000)).groupBy(_._1).mapValues(_.map(p => p._2).distinct.sorted)

In [ ]:
val mergedMap2 = (imd_posting1.toSeq ++ imd_posting2.toSeq)
  .groupBy{case(token,id) => token}
  .mapValues(item => item.flatMap{ case(token,id) => id}.toList)

In [ ]:
mergedMap3

In [ ]:
mergedMap2

In [ ]:
// Frequency Index

case class TfTuple(term: String, doc: String, count: Int)
def tfTuples (docs: Stream[Document]) : Stream[TfTuple] =
    docs.flatMap( d => d.tokens.groupBy(identity)
    .map{ case (tk,lst) => TfTuple(tk, d.name, lst.length) }
val fqIndex : Map[String,List[(String,Int)]] =
  tfTuples(stream).groupBy(_.term)
  .mapValues(_.map(tfT => (tfT.doc, tfT.count)).sorted)

In [ ]:
"b".getBytes

In [ ]:
var test = "a" :: "b" :: "c"

In [ ]:
classpath.add("org.scalaz %% scalaz-core % 7.0.6")

In [ ]:
classpath.add("org.scalaz" %% "scalaz-core" % "7.0.6")



In [ ]:
import scala.collection.immutable.IntMap 

In [ ]:
import scalaz.Scalaz._
//each map contains count of people names
 val names = Map("Sidney" -> 1, "Paul" -> 1, "Jacob" -> 7)
 val moreNames = Map("Sidney" -> 1, "Paul" -> 5, "Nick" -> 2)
 //Money time!
 val mergedMap4 = names.map(p => p._1 -> List(p._2)) |+| moreNames.map(p => p._1 -> List(p._2))

In [ ]:
 //each map contains count of people names
 val names = Map("Sidney" -> 1, "Paul" -> 1, "Jacob" -> 7)
 val moreNames = Map("Sidney" -> 1, "Paul" -> 5, "Nick" -> 2)

In [ ]:
val mergedMap2 = (names.toSeq ++ moreNames.toSeq)
  .groupBy{case(name,amount) => name}
  .mapValues(person => person.map{ case(name,amount) => amount}.toList)

In [ ]:
val pos_i = new indexing.PosIndex(collection.take(100))

In [ ]:
val simple_i = new indexing.SimpleIndex(collection2.take(50000))

In [ ]:
collection2(10).ID

In [ ]:
simple_i.index

In [ ]:
val mb = 1024*1024
val runtime = Runtime.getRuntime
println(s"Used Memory:  " + (runtime.totalMemory - runtime.freeMemory) / mb)
println(s"Free Memory:  " + runtime.freeMemory / mb)
println(s"Total Memory: " + runtime.totalMemory / mb)
println(s"Max Memory:   " + runtime.maxMemory / mb)